In [3]:
from google.colab import files
files.upload()

Saving results_transformers.csv to results_transformers.csv


{'results_transformers.csv': b'model,val_accuracy,val_f1_macro,train_time_sec,infer_time_sec,n_train,n_val\nBERT,0.92,0.91,185.3,2.4,1780,445\nDistilBERT,0.0,0.0,0.0,0.0,1780,445\n'}

In [4]:
import pandas as pd
import numpy as np
import os

PATH = "results_transformers.csv"

if not os.path.exists(PATH):
    raise FileNotFoundError(f"{PATH} introuvable. Vérifie qu'il est bien dans /content (Colab > Files).")

df = pd.read_csv(PATH)

# Conversion numérique
for c in ["val_accuracy","val_f1_macro","train_time_sec","infer_time_sec","n_train","n_val"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Statut run
df["run_status"] = np.where(
    (df["val_accuracy"].fillna(0) == 0) & (df["val_f1_macro"].fillna(0) == 0),
    "FAILED_OR_NOT_RUN",
    "OK"
)

# Tri
df_final = df.sort_values(["val_f1_macro","val_accuracy"], ascending=False).reset_index(drop=True)

# Affichage
df_final


,model,val_accuracy,val_f1_macro,train_time_sec,infer_time_sec,n_train,n_val,run_status
0,BERT,0.92,0.91,185.3,2.4,1780,445,OK
1,DistilBERT,0.00,0.00,0.0,0.0,1780,445,FAILED_OR_NOT_RUN


In [5]:
OUT = "final_comparison_table.csv"
df_final.to_csv(OUT, index=False)
print("Saved:", OUT)

from google.colab import files
files.download(OUT)


Saved: final_comparison_table.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
best = df_final.iloc[0]
print(
    f"Meilleur modèle (F1 macro): {best['model']} | "
    f"F1_macro={best['val_f1_macro']:.2f}, Acc={best['val_accuracy']:.2f} | "
    f"Train={best['train_time_sec']:.1f}s, Infer={best['infer_time_sec']:.1f}s | "
    f"Statut={best['run_status']}."
)

# Note sur les runs échoués
failed = df_final[df_final["run_status"]!="OK"]
if len(failed) > 0:
    print("Runs non valides / non exécutés:", failed["model"].tolist())


Meilleur modèle (F1 macro): BERT | F1_macro=0.91, Acc=0.92 | Train=185.3s, Infer=2.4s | Statut=OK.
Runs non valides / non exécutés: ['DistilBERT']
